## Importations

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_rel

import mne
from mne.channels import find_ch_adjacency, make_1020_channel_selections

## Prepare data

In [ ]:
task = 'LaughterPassive'
subj_list = SUBJ_CLEAN
stage = "epo"

# Select what conditions to compute (str)
cond1 = 'LaughPosed'
cond2 = 'BaselineZero'
cond3 = 'Miss'

if cond1 == 'Good' and 'cond3' == 'Miss' :
    button_cond = True
else :
    button_cond = False
    
conditions = conditions = cond1 + '-' + cond2
condition_list = [cond1, cond2]
picks = "meg" # Select MEG channels

roi = 'MLT'

In [1]:
# Import data

contrasts_all_subject = []
contrasts_all_subject_all_chan = []
evoked_baseline = []
evoked_condition1 = []

for subj in SUBJ_CLEAN :
    print("processing -->", subj)
    
    chan_selected = []
    
    # TODO : change with AR_epochs
    _, path_epochs = get_bids_file(RESULT_PATH, task=task, subj=subj, stage="AR_epo")
    epochs = mne.read_epochs(path_epochs, verbose=None)
    epochs.apply_baseline(baseline=(None, 0))
    
    CHAN = epochs.info['ch_names']
    
    for chan in CHAN  : 
        if roi in chan : 
            chan_selected.append(chan)
    
    epochs_copy = epochs.copy() 
    epochs_copy.pick_channels(chan_selected)
            
    #epochs_copy.filter(1, 20)
    
    # Take nb channel and time
    baseline_data = np.zeros((epochs_copy.get_data().shape[1], epochs_copy.get_data().shape[2]))
    baseline_roi = mne.EvokedArray(baseline_data, epochs_copy.info, tmin=-0.5, comment='baseline')
    
    baseline_data_all_chan = np.zeros((epochs.get_data().shape[1], epochs.get_data().shape[2]))
    baseline_all_chan = mne.EvokedArray(baseline_data_all_chan, epochs.info, tmin=-0.5, comment='baseline')
    
    # Compute button press : Combine cond1 & 2
    if button_cond == True : 
        evoked_cond1_roi = mne.combine_evoked([epochs_copy[cond1].average(), epochs_copy[cond3].average()], weights='nave')
        evoked_cond1_all_chan = mne.combine_evoked([epochs[cond1].average(), epochs[cond3].average()], weights='nave')

    else : 
        
        evoked_cond1_roi = epochs_copy[cond1].average()
        evoked_cond1_all_chan = epochs[cond1].average()

    evoked_baseline.append(baseline_roi) 
    evoked_condition1.append(evoked_cond1_roi)
    
    contrast = mne.combine_evoked([evoked_cond1_roi, baseline_roi], weights=[1, -1])
    contrast_all_chan = mne.combine_evoked([evoked_cond1_all_chan, baseline_all_chan], weights=[1, -1])
    contrast.pick_types(meg=True, ref_meg=False,  exclude='bads')
    
    contrasts_all_subject.append(contrast)
    contrasts_all_subject_all_chan.append(contrast_all_chan)
    
# Combine all subject together
evokeds = {cond1 : evoked_condition1, cond2 : evoked_baseline}

evoked_contrast = mne.combine_evoked(contrasts_all_subject, 'equal')
evoked_contrast_all_chan = mne.combine_evoked(contrasts_all_subject_all_chan, 'equal')


## Perform paired t-test

In [ ]:
# Perform t-test without correction without clusters

time_windows = ((-0.5, 1.5), (0.35, 0.45))
index = ["condition", "epoch", "time"]

# display the EEG data in Pandas format (first 5 rows)
print(epochs.to_data_frame(index=index)[elecs].head())

report = "{elec}, time: {tmin}-{tmax} s; t({df})={t_val:.3f}, p={p:.3f}"
print("\nTargeted statistical test results:")

for tmin, tmax in time_windows:
    cond1_df = long_words.copy().crop(tmin, tmax).to_data_frame(index=index)
    cond2_df = short_words.copy().crop(tmin, tmax).to_data_frame(index=index)
    
    for elec in elecs:
        # extract data
        A = cond1_df[elec].groupby("condition").mean()
        B = cond2_df[elec].groupby("condition").mean() # change that

        # conduct t test
        t, p = ttest_rel(A, B) # Paired t-test

        # display results
        format_dict = dict(
            elec=elec, tmin=tmin, tmax=tmax, df=len(epochs.events) - 2, t_val=t, p=p
        )
        print(report.format(**format_dict))

## Plot

In [ ]:
T_obs, clusters, p_values, _ = cluster_stats # change that

p_accept = 0.001
good_pval_inds = np.where(p_values < p_accept)[0]

# configure variables for visualization
colors = "r",'steelblue'
linestyles = '-', '--'

times = evoked_contrast.times * 1e3

# Prepare mask for topomaps
ch_inds = []

for i, chan in enumerate(evoked_contrast_all_chan.info['ch_names']):
    if roi in evoked_contrast_all_chan.info['ch_names'][i] :    
        ch_inds.append(i)
        
# loop over clusters
for i_clu, clu_idx in enumerate(good_pval_inds):
    
    evoked_contrast_topo = evoked_contrast_all_chan.copy()

    # unpack cluster information, get unique indices
    time_inds = clusters[clu_idx]
    time_inds = np.unique(time_inds)
    sig_times = evoked_contrast.times[time_inds]
    
    # initialize figure
    fig, ax_topo = plt.subplots(1, 1, figsize=(17, 5))
    
    evoked_contrast_topo.crop(tmin=sig_times[0], tmax=sig_times[-1])
    
    # create spatial mask
    mask = np.zeros((evoked_contrast_all_chan.get_data().shape[0], evoked_contrast_topo.times.shape[0]), dtype=bool)
    mask[ch_inds, :] = True

    
    evoked_contrast_topo.plot_topomap(times='peaks', mask=mask, axes=ax_topo, cmap='bwr',
                        cnorm = matplotlib.colors.CenteredNorm(vcenter=0), show=False,
                        colorbar=False, mask_params=dict(markersize=10), extrapolate='head',
                        sphere=(0, 0.019, 0, 0.184))

    image = ax_topo.images[0]

    # remove the title that would otherwise say "0.000 s"
    ax_topo.set_title("")

    # create additional axes (for ERF and colorbar)
    divider = make_axes_locatable(ax_topo)

    # add axes for colorbar
    ax_colorbar = divider.append_axes('right', size='5%', pad=0.05)
    plt.colorbar(image, cax=ax_colorbar)

    # add new axis for time courses and plot time courses
    ax_signals = divider.append_axes('right', size='300%', pad=1.2)

    title = 'Cluster #{0}, (p={1})'.format(i_clu + 1, p_values[clu_idx])

    plot_compare_evokeds(evokeds, title=title,
                    colors=colors, show=False,axes=ax_signals,
                    split_legend=True, truncate_yaxis='auto', combine="mean")

    # plot temporal cluster extent
    ymin, ymax = ax_signals.get_ylim()
    ax_signals.fill_betweenx((ymin, ymax), sig_times[0], sig_times[-1],
                        color='orange', alpha=0.3)
    # clean up viz
    mne.viz.tight_layout(fig=fig)
    fig.subplots_adjust(bottom=.05)
    
    del evoked_contrast_topo